In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\jarod\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
df_cvs = pd.read_csv('../../data/raw_data/datasets/cv_raw_V0')

**Eliminar caracter especial '\n'**

In [3]:
def delete_line_break(df):
    df_copy = df.copy()
    df_copy.iloc[:, 1] = df_copy.iloc[:, 1].apply(lambda x: x.replace('\n', ' '))
    return df_copy

In [4]:
df_cvs=delete_line_break(df_cvs)

**Modelado**

In [5]:
# Carga del modelo y tokenizador
model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [6]:
# Tokenizamos los textos
tokenized_inputs = tokenizer(df_cvs['cv'].to_list(), padding=True, 
                             truncation=True, max_length=768, return_tensors="tf")

In [7]:
# Extraemos el resultado en funcion de las representaciones de MiniLM
outputs = model(tokenized_inputs)

In [8]:
# Capturamos los embeddings
embeddings = outputs.last_hidden_state

In [9]:
# Normaliza los vectores de embedding para asegurarte de que tengan longitud unitaria
normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

In [10]:
def calculate_cosine(mean_embedding_1,mean_embedding_2):
    similarity_score = cosine_similarity([mean_embedding_1], [mean_embedding_2])[0, 0]
    return similarity_score

In [11]:
def similitud_cv(cv_1, cv_2):
    
    # Calcular la media de los embeddings
    mean_embedding_1 = np.mean(normalized_embeddings[cv_1], axis=0)
    mean_embedding_2 = np.mean(normalized_embeddings[cv_2], axis=0)

    # Calcular la similitud coseno entre los embeddings
    similarity_score = calculate_cosine(mean_embedding_1,mean_embedding_2)

    # Mostrar en pantalla resultados
    print(f"La similaridad entre el CV {cv_1} y {cv_2} es de:", similarity_score)
    print('\n------------')
    print(f'CV con id {cv_1}')
    print(df_cvs['cv'][cv_1])
    print('\n------------')
    print(f'CV con id {cv_2}')
    print(df_cvs['cv'][cv_2])

In [20]:
similitud_cv(10, 23)

La similaridad entre el CV 10 y 23 es de: 0.24017674

------------
CV con id 10
Enfoque en la experiencia en Cisco  Nombre: Juan Pérez  Resumen: Ingeniero de Redes con más de 5 años de experiencia en diseño, implementación y mantenimiento de redes Cisco. Especializado en enrutamiento y conmutación, seguridad de red y administración de dispositivos Cisco.  Experiencia Laboral:  Ingeniero de Redes en ABC Solutions (2018 - Presente) Lideró el diseño e implementación de una red empresarial basada en Cisco para una empresa de manufactura. Realizó la configuración de dispositivos Cisco ISR, switches Catalyst y firewalls ASA. Técnico de Redes en XYZ Networking (2015 - 2018) Proporcionó soporte técnico y mantenimiento preventivo a la infraestructura de red de clientes, incluida la resolución de problemas y actualizaciones de firmware. Educación:  Licenciatura en Ingeniería de Telecomunicaciones, Universidad Nacional (2014) Certificaciones:  Cisco Certified Network Professional (CCNP) Cisco Cer